In [ ]:
# Code adapted for OpenAI GPT API
#Note: the code will run best when executing each section separately

# 1. GPT identify federal securities laws
import os
import openai
import pandas as pd
import re
from typing import List, Dict

def get_securities_laws(conversation_history=None):
    # Initialize the OpenAI client
    client = openai.OpenAI(
        api_key="enter API here"
    )
    
    # Initial prompt 
    initial_content = """Your task is to identify and compile a comprehensive database of at least 100 federal securities 
    laws. Securities regulation is the field of U.S. law that covers transactions and other dealings with securities. 
    Securities laws aim at ensuring that investors receive accurate and necessary information regarding the type and value
    of the interest under consideration for purchase.

IMPORTANT: Only identify new securities regulations. Exclude amendments, updates, or revisions to existing rules. 
Focus on major new laws only.

IMPORTANT: Do not include laws with titles containing the following words: "Amendment", "Update", or "Revision"

The goal is to create a dataset that captures the following key details for each law. 

Please follow these guidelines:

Data Fields to Collect:
• Date: The announcement or implementation date of the law (use YYYY-MM-DD format).
• Regulation Title or Name: The official name or designation of the regulatory change.
• Regulatory Body/Authority: The government entity responsible for the law.
• Description: A brief overview of the law, including key provisions and the rationale behind it.
• Impact: The potential or observed effects on industries, markets, or stakeholders.
•Litigation Risk: Is this law related to the risk of litigation against managers? By risk of litigation we mean the probability that a manager will be sued or face legal action because of this law. Answer this question with Yes or No. If yes, label the entry "Litigation Risk".
•Corporate Governance: Is this law related to corporate governance of firms? Corporate governance refers to the internal monitoring system charged with overseeing managers and commonly focuses on matters such as board independence or insider trading policy. Answer this question with Yes or No.If yes, label the entry "Corporate Governance".
•Proprietary Costs: Is this law related to proprietary costs of firms? By proprietary costs, we mean costs that result from the disclosure of information to competitors which could harm a firm's competitive position. Answer this question with Yes or No.If yes, label the entry "Proprietary Costs".
•Information Asymmetry: Is this law related to information asymmetry between owners and managers? By information asymmetry we mean that one party has more or better information than the other party. Answer this question with Yes or No. If yes, label the entry "Information Asymmetry".
•Unsophisticated Investors: Is the law related to protecting unsophisticated investors? By unsophisticated investors, we mean investors that are either new to investing or are not well informed. Answer this question with Yes or No. If yes, label the entry "Unsophisticated Investors".
•Equity Issuance in Public vs. Private Markets: Is this law related to the costs and benefits of issuing equity in public versus private markets? Answer this question with Yes or No. If yes, label the entry "Equity Issuance in Public vs. Private Markets".
•Reputation Risk: Is this law related to the reputation of firm managers? By of firm manager, we mean the career prospects and prestige of an individual manager. Answer this question with Yes or No. If yes, label the entry "Reputation Risk".

• References: Links to official documents or credible news sources.

Requirements:
• Scope: Cover as many laws as possible that were announced or implemented in the last 25 years.
• Consistency: Ensure uniform formatting for all entries in the dataset.
• Dates must be in YYYY-MM-DD format (e.g., 2002-07-30).

Output:
Provide data in a tabular format with rows for each law and columns for the data fields listed above. 
Use credible, authoritative sources such as government websites, legal databases, academic journals, or credible news sources.
Do not include duplicate laws.
"""

    try:
        if conversation_history:
            messages = conversation_history
        else:
            messages = [{
                "role": "user",
                "content": initial_content
            }]

        print("Making API call to GPT-4.1...")
        response = client.chat.completions.create(
            model="gpt-4.1",  
            messages=messages,
            max_tokens=8000,  # GPT-4.1 uses max_tokens, not max_completion_tokens
            temperature=0.5 
        )
        
        print(f"Raw response object: {response}")
        print(f"Response choices: {response.choices}")
        print(f"First choice: {response.choices[0] if response.choices else 'No choices'}")
        
        response_content = response.choices[0].message.content
        print(f"Response content type: {type(response_content)}")
        print(f"Response content: {repr(response_content)}")
        print(f"API call successful! Response length: {len(response_content) if response_content else 0} characters")
        
        return response_content, messages + [
            {"role": "assistant", "content": response_content}
        ]
    except Exception as e:
        print(f"Error making API call: {e}")
        return None, messages if conversation_history else []


def add_follow_up_prompt(conversation_history, follow_up_prompt):
    """Add a follow-up prompt to the conversation history"""
    return conversation_history + [{"role": "user", "content": follow_up_prompt}]

def standardize_date(date_str):
    """Attempt to standardize date format to YYYY-MM-DD"""
    try:
        # Convert to datetime and then back to string in desired format
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except:
        # If conversion fails, return original string
        return date_str

def parse_table_fallback(response_text: str) -> pd.DataFrame:
    """Fallback parser for when Claude returns table format instead of numbered format."""
    print("Debugging table parsing...")
    
    lines = response_text.split('\n')
    table_lines = [line.strip() for line in lines if line.strip().startswith('|') and len(line.strip()) > 5]
    
    print(f"Found {len(table_lines)} table lines")
    
    if len(table_lines) < 2:
        print("Not enough table lines found")
        return pd.DataFrame()
    
    # Remove separator lines (containing ---)
    data_lines = [line for line in table_lines if '---' not in line]
    print(f"Found {len(data_lines)} data lines (after removing separators)")
    
    if len(data_lines) < 2:
        print("Not enough data lines after removing separators")
        return pd.DataFrame()
    
    # Parse header line
    header_line = data_lines[0]
    raw_headers = header_line.split('|')
    headers = [col.strip() for col in raw_headers if col.strip()]
    
    print(f"Original headers ({len(headers)}): {headers}")
    
    # Parse data rows
    data_rows = []
    for i, line in enumerate(data_lines[1:], 1):
        raw_columns = line.split('|')
        columns = [col.strip() for col in raw_columns if col.strip()]
        
        if len(columns) == len(headers):
            data_rows.append(columns)
            print(f"Row {i}: ✓ Added ({len(columns)} columns)")
        else:
            print(f"Row {i}: ✗ Skipped - {len(columns)} columns vs {len(headers)} headers")
    
    print(f"Successfully parsed {len(data_rows)} data rows")
    
    if not data_rows:
        print("No valid data rows found")
        return pd.DataFrame()
    
    # Create DataFrame with original headers first
    df = pd.DataFrame(data_rows, columns=headers)
    print(f"Created DataFrame with columns: {list(df.columns)}")

    # Make the mapping more robust for title column
    for col in df.columns:
        if 'title' in col.lower() or 'name' in col.lower():
            df = df.rename(columns={col: 'Regulation Title'})
            print(f"Mapped column '{col}' to 'Regulation Title'")
            break
            
    # Now standardize the column names to match expected format
    # Create a mapping from the table headers to standard column names
    standard_columns = {
        'Date': 'Date',
        'Regulation Title/Name': 'Regulation Title', 
        'Regulatory Body': 'Regulatory Body',
        'Regulatory Body/Authority': 'Regulatory Body',  # Handle variation
        'Description': 'Description',
        'Impact': 'Impact',
        'Litigation Risk': 'Litigation Risk',
        'Corporate Governance': 'Corporate Governance', 
        'Proprietary Costs': 'Proprietary Costs',
        'Information Asymmetry': 'Information Asymmetry',
        'Unsophisticated Investors': 'Unsophisticated Investors',
        'Equity Issuance Public vs Private': 'Equity Issuance',
        'Equity Issuance in Public vs. Private Markets': 'Equity Issuance',  # Handle variation
        'Reputation Risk': 'Reputation Risk',
        'References': 'References'
    }
    
    # Rename columns using the mapping
    df_renamed = df.rename(columns=standard_columns)
    
    # Ensure all required columns exist
    required_columns = [
        'Date', 'Regulation Title', 'Regulatory Body', 'Description', 'Impact',
        'Litigation Risk', 'Corporate Governance', 'Proprietary Costs', 
        'Information Asymmetry', 'Unsophisticated Investors', 'Equity Issuance',
        'Reputation Risk', 'References'
    ]
    
    # Add missing columns with None values
    for col in required_columns:
        if col not in df_renamed.columns:
            print(f"Adding missing column: {col}")
            df_renamed[col] = None
    
    # Select only the required columns in the correct order
    final_df = df_renamed[required_columns].copy()
    
    # Standardize date format
    if 'Date' in final_df.columns:
        print("Standardizing dates...")
        final_df['Date'] = final_df['Date'].apply(lambda x: standardize_date(x) if pd.notna(x) and str(x).strip() else x)
    
    # Clean up any completely empty rows
    final_df = final_df.dropna(how='all')
    
    print(f"Final DataFrame: {len(final_df)} rows x {len(final_df.columns)} columns")
    print(f"Final columns: {list(final_df.columns)}")
    
    return final_df

def parse_bullet_format(response_text: str) -> pd.DataFrame:
    """Parse GPT response in numbered + markdown bullet format."""
    import re
    entries = []
    current_entry = {}
    entry_number = None

    lines = [line.strip() for line in response_text.split("\n") if line.strip()]

    for line in lines:
        # Detect new numbered law like "4." or "12."
        number_match = re.match(r'^(\d+)\.', line)
        if number_match:
            if current_entry:
                entries.append(current_entry)
            current_entry = {}
            entry_number = number_match.group(1)
            continue

        # Detect bullet fields like "- **Date**: 1996-10-11"
        bullet_match = re.match(r'^-\s*\*\*(.+?)\*\*:\s*(.+)', line)
        if bullet_match and current_entry is not None:
            key, value = bullet_match.groups()
            key = key.strip()
            value = value.strip()

            key_mapping = {
                "Date": "Date",
                "Title": "Regulation Title",
                "Authority": "Regulatory Body",
                "Description": "Description",
                "Impact": "Impact",
                "Litigation Risk": "Litigation Risk",
                "Corporate Governance": "Corporate Governance",
                "Proprietary Costs": "Proprietary Costs",
                "Information Asymmetry": "Information Asymmetry",
                "Unsophisticated Investors": "Unsophisticated Investors",
                "Equity Issuance": "Equity Issuance",
                "Reputation Risk": "Reputation Risk",
                "References": "References"
            }

            if key in key_mapping:
                current_entry[key_mapping[key]] = value

    # Append last entry
    if current_entry:
        entries.append(current_entry)

    if not entries:
        return pd.DataFrame()

    df = pd.DataFrame(entries)

    # Ensure all required columns
    required_columns = [
        'Date', 'Regulation Title', 'Regulatory Body', 'Description', 'Impact',
        'Litigation Risk', 'Corporate Governance', 'Proprietary Costs',
        'Information Asymmetry', 'Unsophisticated Investors', 'Equity Issuance',
        'Reputation Risk', 'References'
    ]
    for col in required_columns:
        if col not in df.columns:
            df[col] = None

    return df[required_columns]
    
def parse_response_to_dataframe(response_text: str) -> pd.DataFrame:
    """Parse the response text into a pandas DataFrame."""
    print("\nParsing response...")
    
    # First try the original numbered format parsing
    data = []
    current_entry = None
    entry_number = None
    
    lines = [line.strip() for line in response_text.split('\n') if line.strip()]
    
    for line in lines:
        number_match = re.match(r'^\*?\*?(\d+)\.\*?\*?', line)
        if number_match:
            if current_entry and len(current_entry) > 0:
                if 'Regulation Title' not in current_entry and entry_number:
                    current_entry['Regulation Title'] = f"Law {entry_number}"
                data.append(current_entry)
            current_entry = {}
            entry_number = number_match.group(1)
            continue
            
        if ':' in line and current_entry is not None:
            key, value = [x.strip() for x in line.split(':', 1)]
            
            key_mapping = {
                'Date': 'Date',
                'Title': 'Regulation Title',
                'Authority': 'Regulatory Body',
                'Description': 'Description',
                'Impact': 'Impact',
                'Litigation Risk': 'Litigation Risk',
                'Corporate Governance': 'Corporate Governance',
                'Proprietary Costs': 'Proprietary Costs',
                'Information Asymmetry': 'Information Asymmetry',
                'Unsophisticated Investors': 'Unsophisticated Investors',
                'Equity Issuance': 'Equity Issuance',
                'Reputation Risk': 'Reputation Risk',
                'References': 'References'
            }
            
            if key in key_mapping:
                column_name = key_mapping[key]
                if column_name == 'Date':
                    current_entry[column_name] = standardize_date(value)
                else:
                    current_entry[column_name] = value.strip()

    if current_entry and len(current_entry) > 0:
        if 'Regulation Title' not in current_entry and entry_number:
            current_entry['Regulation Title'] = f"Law {entry_number}"
        data.append(current_entry)

    print(f"\nFound {len(data)} entries in numbered format")

    if data:
        df = pd.DataFrame(data)
        required_columns = [
            'Date', 'Regulation Title', 'Regulatory Body', 'Description', 'Impact',
            'Litigation Risk', 'Corporate Governance', 'Proprietary Costs',
            'Information Asymmetry', 'Unsophisticated Investors', 'Equity Issuance',
            'Reputation Risk', 'References'
        ]
        for col in required_columns:
            if col not in df.columns:
                print(f"Adding missing column: {col}")
                df[col] = None
        df['Regulation Title'] = df['Regulation Title'].fillna('Unknown')
        df['Regulation Title'] = df['Regulation Title'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
        df['dedup_key'] = df.apply(lambda row: f"{row['Date']}_{row['Regulation Title']}", axis=1)
        df = df.drop_duplicates(subset=['dedup_key'], keep='first')
        df = df.drop('dedup_key', axis=1)
        df = df[required_columns]
        print(f"Created DataFrame with {len(df)} rows")
        return df.copy()

    # If numbered format failed, try table format
    print("Numbered format parsing failed, trying table format...")
    df = parse_table_fallback(response_text)
    if not df.empty:
        print(f"Table format parsing succeeded with {len(df)} rows")
        return df

    # If table format failed, try bullet format
    print("Table format parsing failed, trying bullet format...")
    df = parse_bullet_format(response_text)
    if not df.empty:
        print(f"Bullet format parsing succeeded with {len(df)} rows")
        return df

    # All methods failed
    print("All parsing methods failed - no valid data to create DataFrame")
    return pd.DataFrame()
                
def compile_all_responses() -> pd.DataFrame:
    """Compile multiple API responses into a single DataFrame."""
    all_responses = []
    conversation_history = None

    # Get initial response
    initial_response, conversation_history = get_securities_laws()
    if initial_response:
        print("\nInitial response:")
        print("="*50)
        print(initial_response)
        print("="*50)
        print(f"Response length: {len(initial_response)} characters")
        all_responses.append(initial_response)
    else:
        print("ERROR: No initial response received from API!")
        return pd.DataFrame()

    # Follow-up prompts for additional laws
    follow_up_prompts = [
        """Starting with number {last_num}, list 20 more federal securities laws using this exact format for each:
Date: YYYY-MM-DD
Title: [title]
Authority: [body]
Description: [brief]
Impact: [impact]
Litigation Risk: Yes/No
Corporate Governance: Yes/No
Proprietary Costs: Yes/No
Information Asymmetry: Yes/No
Unsophisticated Investors: Yes/No
Equity Issuance: Yes/No
Reputation Risk: Yes/No
References: [link]""",

        "Continue from number {last_num}. Provide 20 more laws using the exact same format.",
        
        "List 20 more laws starting at number {last_num}. Use the same format.",
        
        "Add 20 more laws beginning with number {last_num}. Keep the same format.",
        
        "Provide 20 more laws from number {last_num}. Same format.",
        
        "Recall that you have to identify at least 100 federal securities laws. Recall securities regulation is the field of U.S. law that covers transactions and other dealings with securities. Securities laws aim at ensuring that investors receive accurate and necessary information regarding the type and value of the interest under consideration for purchase."
    ]
    last_num = len(parse_response_to_dataframe(initial_response)) + 1
    
    for i, prompt_template in enumerate(follow_up_prompts, 1):
        prompt = prompt_template.format(last_num=last_num)
        conversation_history = add_follow_up_prompt(conversation_history, prompt)
        response, conversation_history = get_securities_laws(conversation_history)
        
        if response:
            print(f"\nFollow-up response {i}:")
            print(response)
            all_responses.append(response)
            df = parse_response_to_dataframe(response)
            last_num += len(df)
            
    # Parse all responses into DataFrames and concatenate
    dfs = []
    for response in all_responses:
        df = parse_response_to_dataframe(response)
        if not df.empty:
            dfs.append(df)

    if not dfs:
        print("No valid data frames were created!")
        return pd.DataFrame()

    # Concatenate all DataFrames
    final_df = pd.concat(dfs, ignore_index=True)

    # Remove duplicates using multiple fields to better identify unique laws
    final_df['Title_clean'] = final_df['Regulation Title'].fillna('').str.lower().str.strip()
    final_df['Description_clean'] = final_df['Description'].fillna('').str.lower().str.strip()
    
    # Create composite key for deduplication
    final_df['dedup_key'] = final_df.apply(
        lambda row: f"{row['Date']}_{row['Title_clean']}_{row['Description_clean'][:50]}", 
        axis=1
    )
    
    # Remove duplicates and cleanup
    final_df = final_df.drop_duplicates(subset=['dedup_key'], keep='first')
    final_df = final_df.drop(['Title_clean', 'Description_clean', 'dedup_key'], axis=1)

    # Sort by date
    try:
        final_df['DateSort'] = pd.to_datetime(final_df['Date'], errors='coerce')
        final_df = final_df.dropna(subset=['DateSort'])
        final_df = final_df.sort_values('DateSort', ascending=False)
        final_df = final_df.drop('DateSort', axis=1)
    except Exception as e:
        print(f"Warning: Could not sort by date due to: {e}")
        print("Problematic dates:")
        print(final_df['Date'].value_counts())

    # Return the final DataFrame
    return final_df

if __name__ == "__main__":
    # Compile all responses into a DataFrame
    df = compile_all_responses()
    
    if df.empty:
        print("\nError: No data was collected!")
    else:
        # Display basic statistics
        print(f"\nTotal number of unique laws: {len(df)}")
        print("\nMost recent laws:")
        print(df.head().to_string())
        
        # Save to CSV
        output_path = 'enter file path here'
        df.to_csv(output_path, index=False)
        print(f"\nDatabase saved to: {output_path}")
        
# 2. Add column for Year 

import pandas as pd

df=pd.read_csv('enter file path here')

# Clean parentheses and dashes from text columns
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].str.replace('(', '').str.replace(')', '').str.replace('-', '')
    
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df

#Excluding years prior to 2002 2020 and 2021 since we don't have forecast data. We also exclude years 2018, 2019 from law file because we need 2 years after and we have data
#up to 2019
filtered_df = df[~df['Year'].isin([1986, 1987, 1988, 1989,1990, 1991, 1992, 1993, 1994, 1995, 1996,
                                   1997, 1998, 1999, 2000, 2001,2018, 2019, 2020, 2021, 2022, 2023, 2024])]

filtered_df_with_titles = filtered_df.dropna(subset=["Regulatory Body"])

filtered_df_with_titles.to_csv('enter file path here')

In [ ]:
# Code adapted for Google Gemini API
#Note: the code will run best when executing each section separately

# 1. Gemini identify federal securities laws
import os
import pandas as pd
import re
from typing import List, Dict

import google.generativeai as genai


# ---------------------------
# Gemini client + conversation helpers
# ---------------------------
#this function is needed for follow-up prompts otherwise gemini will not remember the converstation 
def _to_gemini_history(conv_hist):
    """
    Convert OpenAI-style:
      [{"role":"user"|"assistant", "content":"..."}]
    to Gemini chat history:
      [{"role":"user"|"model", "parts":[ "..."]}, ...]
    """
    if not conv_hist:
        return []
    role_map = {"user": "user", "assistant": "model"}
    history = []
    for m in conv_hist:
        r = role_map.get(m.get("role"), "user")
        history.append({"role": r, "parts": [m.get("content", "")]})
    return history


def get_securities_laws(conversation_history=None):
    # Configure Gemini with your API key 
    genai.configure(api_key="enter API here")  

    # Initial prompt 
    initial_content = """Your task is to identify and compile a comprehensive database of at least 100 federal securities 
    laws. Securities regulation is the field of U.S. law that covers transactions and other dealings with securities. 
    Securities laws aim at ensuring that investors receive accurate and necessary information regarding the type and value
    of the interest under consideration for purchase.

IMPORTANT: Only identify new securities regulations. Exclude amendments, updates, or revisions to existing rules. 
Focus on major new laws only.

IMPORTANT: Do not include laws with titles containing the following words: "Amendment", "Update", or "Revision"

The goal is to create a dataset that captures the following key details for each law. 

Please follow these guidelines:

Data Fields to Collect:
• Date: The announcement or implementation date of the law (use YYYY-MM-DD format).
• Regulation Title or Name: The official name or designation of the regulatory change.
• Regulatory Body/Authority: The government entity responsible for the law.
• Description: A brief overview of the law, including key provisions and the rationale behind it.
• Impact: The potential or observed effects on industries, markets, or stakeholders.
•Litigation Risk: Is this law related to the risk of litigation against managers? By risk of litigation we mean the probability that a manager will be sued or face legal action because of this law. Answer this question with Yes or No. If yes, label the entry "Litigation Risk".
•Corporate Governance: Is this law related to corporate governance of firms? Corporate governance refers to the internal monitoring system charged with overseeing managers and commonly focuses on matters such as board independence or insider trading policy. Answer this question with Yes or No.If yes, label the entry "Corporate Governance".
•Proprietary Costs: Is this law related to proprietary costs of firms? By proprietary costs, we mean costs that result from the disclosure of information to competitors which could harm a firm's competitive position. Answer this question with Yes or No.If yes, label the entry "Proprietary Costs".
•Information Asymmetry: Is this law related to information asymmetry between owners and managers? By information asymmetry we mean that one party has more or better information than the other party. Answer this question with Yes or No. If yes, label the entry "Information Asymmetry".
•Unsophisticated Investors: Is the law related to protecting unsophisticated investors? By unsophisticated investors, we mean investors that are either new to investing or are not well informed. Answer this question with Yes or No. If yes, label the entry "Unsophisticated Investors".
•Equity Issuance in Public vs. Private Markets: Is this law related to the costs and benefits of issuing equity in public versus private markets? Answer this question with Yes or No. If yes, label the entry "Equity Issuance in Public vs. Private Markets".
•Reputation Risk: Is this law related to the reputation of firm managers? By of firm manager, we mean the career prospects and prestige of an individual manager. Answer this question with Yes or No. If yes, label the entry "Reputation Risk".

• References: Links to official documents or credible news sources.

Requirements:
• Scope: Cover as many laws as possible that were announced or implemented in the last 25 years.
• Consistency: Ensure uniform formatting for all entries in the dataset.
• Dates must be in YYYY-MM-DD format (e.g., 2002-07-30).

Output:
Provide data in a tabular format with rows for each law and columns for the data fields listed above. 
Use credible, authoritative sources such as government websites, legal databases, academic journals, or credible news sources.
Do not include duplicate laws.
"""

    try:
        model = genai.GenerativeModel("gemini-2.5-pro")
        generation_config = {
            "temperature": 0.5,
            "max_output_tokens": 8000,
        }

        if conversation_history:
            # ---- EDIT 1: send the latest user prompt, not empty content ----
            last_user_msg = ""
            if conversation_history and conversation_history[-1].get("role") == "user":
                last_user_msg = conversation_history[-1].get("content", "")
                history_without_last = conversation_history[:-1]
            else:
                history_without_last = conversation_history

            chat = model.start_chat(history=_to_gemini_history(history_without_last))
            print("Making API call to Gemini (chat.send_message)...")
            if not last_user_msg:
                last_user_msg = "Continue."
            resp = chat.send_message(last_user_msg, generation_config=generation_config)

            response_content = getattr(resp, "text", "") or ""
            print(f"Raw response object: {resp}")
            print(f"Response content type: {type(response_content)}")
            print(f"Response content: {repr(response_content)}")
            print(f"API call successful! Response length: {len(response_content)} characters")

            updated_history = conversation_history + [{"role": "assistant", "content": response_content}]
            return response_content, updated_history

        else:
            # First turn (no history yet)
            print("Making API call to Gemini (generate_content)...")
            resp = model.generate_content(initial_content, generation_config=generation_config)
            response_content = getattr(resp, "text", "") or ""
            print(f"Raw response object: {resp}")
            print(f"Response content type: {type(response_content)}")
            print(f"Response content: {repr(response_content)}")
            print(f"API call successful! Response length: {len(response_content)} characters")
            messages = [
                {"role": "user", "content": initial_content},
                {"role": "assistant", "content": response_content},
            ]
            return response_content, messages

    except Exception as e:
        print(f"Error making Gemini API call: {e}")
        return None, conversation_history if conversation_history else []


def add_follow_up_prompt(conversation_history, follow_up_prompt):
    """Add a follow-up prompt to the conversation history"""
    return conversation_history + [{"role": "user", "content": follow_up_prompt}]


def standardize_date(date_str):
    """Attempt to standardize date format to YYYY-MM-DD"""
    try:
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except Exception:
        return date_str


# ---------------------------
# Parsers
# ---------------------------

def parse_table_fallback(response_text: str) -> pd.DataFrame:
    """Fallback parser for when the model returns a Markdown table."""
    print("Debugging table parsing...")

    lines = response_text.split('\n')
    table_lines = [line.strip() for line in lines if line.strip().startswith('|') and len(line.strip()) > 5]

    print(f"Found {len(table_lines)} table lines")

    if len(table_lines) < 2:
        print("Not enough table lines found")
        return pd.DataFrame()

    # Remove separator lines (containing ---)
    data_lines = [line for line in table_lines if '---' not in line]
    print(f"Found {len(data_lines)} data lines (after removing separators)")

    if len(data_lines) < 2:
        print("Not enough data lines after removing separators")
        return pd.DataFrame()

    # Parse header line
    header_line = data_lines[0]
    raw_headers = header_line.split('|')
    headers = [col.strip() for col in raw_headers if col.strip()]

    print(f"Original headers ({len(headers)}): {headers}")

    # Parse data rows
    data_rows = []
    for i, line in enumerate(data_lines[1:], 1):
        raw_columns = line.split('|')
        columns = [col.strip() for col in raw_columns if col.strip()]
        if len(columns) == len(headers):
            data_rows.append(columns)
            print(f"Row {i}: ✓ Added ({len(columns)} columns)")
        else:
            print(f"Row {i}: ✗ Skipped - {len(columns)} columns vs {len(headers)} headers")

    print(f"Successfully parsed {len(data_rows)} data rows")

    if not data_rows:
        print("No valid data rows found")
        return pd.DataFrame()

    df = pd.DataFrame(data_rows, columns=headers)
    print(f"Created DataFrame with columns: {list(df.columns)}")

    # ---- EDIT 2: Standardize column names (add 'Regulation Title or Name' and fallback copy) ----
    standard_columns = {
        'Date': 'Date',
        'Regulation Title/Name': 'Regulation Title',
        'Regulation Title or Name': 'Regulation Title',  # new variant
        'Regulatory Body': 'Regulatory Body',
        'Regulatory Body/Authority': 'Regulatory Body',
        'Description': 'Description',
        'Impact': 'Impact',
        'Litigation Risk': 'Litigation Risk',
        'Corporate Governance': 'Corporate Governance',
        'Proprietary Costs': 'Proprietary Costs',
        'Information Asymmetry': 'Information Asymmetry',
        'Unsophisticated Investors': 'Unsophisticated Investors',
        'Equity Issuance Public vs Private': 'Equity Issuance',
        'Equity Issuance in Public vs. Private Markets': 'Equity Issuance',
        'Reputation Risk': 'Reputation Risk',
        'References': 'References'
    }

    df_renamed = df.rename(columns=standard_columns)

    # If "Regulation Title" still missing, copy from any plausible original header
    if 'Regulation Title' not in df_renamed.columns:
        for alt in ['Regulation Title or Name', 'Regulation Title/Name', 'Regulation']:
            if alt in df.columns:
                df_renamed['Regulation Title'] = df[alt]
                break

    required_columns = [
        'Date', 'Regulation Title', 'Regulatory Body', 'Description', 'Impact',
        'Litigation Risk', 'Corporate Governance', 'Proprietary Costs',
        'Information Asymmetry', 'Unsophisticated Investors', 'Equity Issuance',
        'Reputation Risk', 'References'
    ]

    for col in required_columns:
        if col not in df_renamed.columns:
            print(f"Adding missing column: {col}")
            df_renamed[col] = None

    final_df = df_renamed[required_columns].copy()

    # Standardize date format
    if 'Date' in final_df.columns:
        print("Standardizing dates...")
        final_df['Date'] = final_df['Date'].apply(
            lambda x: standardize_date(x) if pd.notna(x) and str(x).strip() else x
        )

    final_df = final_df.dropna(how='all')

    print(f"Final DataFrame: {len(final_df)} rows x {len(final_df.columns)} columns")
    print(f"Final columns: {list(final_df.columns)}")

    return final_df


def parse_bullet_format(response_text: str) -> pd.DataFrame:
    """Parse response in numbered + markdown bullet format."""
    entries = []
    current_entry = {}
    entry_number = None

    lines = [line.strip() for line in response_text.split("\n") if line.strip()]

    for line in lines:
        # Detect new numbered law like "4." or "12."
        number_match = re.match(r'^(\d+)\.', line)
        if number_match:
            if current_entry:
                entries.append(current_entry)
            current_entry = {}
            entry_number = number_match.group(1)
            continue

        # Detect bullet fields like "- **Date**: 1996-10-11"
        bullet_match = re.match(r'^-\s*\*\*(.+?)\*\*:\s*(.+)', line)
        if bullet_match and current_entry is not None:
            key, value = bullet_match.groups()
            key = key.strip()
            value = value.strip()

            key_mapping = {
                "Date": "Date",
                "Title": "Regulation Title",
                "Authority": "Regulatory Body",
                "Description": "Description",
                "Impact": "Impact",
                "Litigation Risk": "Litigation Risk",
                "Corporate Governance": "Corporate Governance",
                "Proprietary Costs": "Proprietary Costs",
                "Information Asymmetry": "Information Asymmetry",
                "Unsophisticated Investors": "Unsophisticated Investors",
                "Equity Issuance": "Equity Issuance",
                "Reputation Risk": "Reputation Risk",
                "References": "References"
            }

            if key in key_mapping:
                current_entry[key_mapping[key]] = value

    if current_entry:
        entries.append(current_entry)

    if not entries:
        return pd.DataFrame()

    df = pd.DataFrame(entries)

    required_columns = [
        'Date', 'Regulation Title', 'Regulatory Body', 'Description', 'Impact',
        'Litigation Risk', 'Corporate Governance', 'Proprietary Costs',
        'Information Asymmetry', 'Unsophisticated Investors', 'Equity Issuance',
        'Reputation Risk', 'References'
    ]
    for col in required_columns:
        if col not in df.columns:
            df[col] = None

    return df[required_columns]


def parse_response_to_dataframe(response_text: str) -> pd.DataFrame:
    """Parse the response text into a pandas DataFrame."""
    print("\nParsing response...")
    
    # First try the updated bold markdown format parsing
    data = []
    current_entry = None
    entry_number = None
    
    lines = [line.strip() for line in response_text.split('\n') if line.strip()]
    
    for line in lines:
        # Updated regex to handle bold markdown format: **24.**
        number_match = re.match(r'^\*\*(\d+)\.\*\*', line)
        if number_match:
            if current_entry and len(current_entry) > 0:
                if 'Regulation Title' not in current_entry and entry_number:
                    current_entry['Regulation Title'] = f"Law {entry_number}"
                data.append(current_entry)
            current_entry = {}
            entry_number = number_match.group(1)
            continue
            
        # Updated to handle bold markdown fields: **Date:** value
        if ':' in line and current_entry is not None:
            # Handle both **Field:** and Field: formats
            if line.startswith('**') and ':**' in line:
                # Bold markdown format: **Date:** value
                key_value = line.split(':**', 1)
                key = key_value[0].replace('**', '').strip()
                value = key_value[1].strip() if len(key_value) > 1 else ''
            elif ':' in line:
                # Regular format: Date: value
                key, value = [x.strip() for x in line.split(':', 1)]
            else:
                continue
            
            key_mapping = {
                'Date': 'Date',
                'Title': 'Regulation Title',
                'Authority': 'Regulatory Body',
                'Description': 'Description',
                'Impact': 'Impact',
                'Litigation Risk': 'Litigation Risk',
                'Corporate Governance': 'Corporate Governance',
                'Proprietary Costs': 'Proprietary Costs',
                'Information Asymmetry': 'Information Asymmetry',
                'Unsophisticated Investors': 'Unsophisticated Investors',
                'Equity Issuance': 'Equity Issuance',
                'Equity Issuance in Public vs. Private Markets': 'Equity Issuance',
                'Reputation Risk': 'Reputation Risk',
                'References': 'References'
            }
            
            if key in key_mapping:
                column_name = key_mapping[key]
                if column_name == 'Date':
                    current_entry[column_name] = standardize_date(value)
                else:
                    current_entry[column_name] = value.strip()

    # Add the last entry
    if current_entry and len(current_entry) > 0:
        if 'Regulation Title' not in current_entry and entry_number:
            current_entry['Regulation Title'] = f"Law {entry_number}"
        data.append(current_entry)

    print(f"\nFound {len(data)} entries in bold markdown format")

    if data:
        df = pd.DataFrame(data)
        required_columns = [
            'Date', 'Regulation Title', 'Regulatory Body', 'Description', 'Impact',
            'Litigation Risk', 'Corporate Governance', 'Proprietary Costs',
            'Information Asymmetry', 'Unsophisticated Investors', 'Equity Issuance',
            'Reputation Risk', 'References'
        ]
        for col in required_columns:
            if col not in df.columns:
                print(f"Adding missing column: {col}")
                df[col] = None
        df['Regulation Title'] = df['Regulation Title'].fillna('Unknown')
        df['Regulation Title'] = df['Regulation Title'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
        df['dedup_key'] = df.apply(lambda row: f"{row['Date']}_{row['Regulation Title']}", axis=1)
        df = df.drop_duplicates(subset=['dedup_key'], keep='first')
        df = df.drop('dedup_key', axis=1)
        df = df[required_columns]
        print(f"Created DataFrame with {len(df)} rows")
        return df.copy()

    # If bold markdown format failed, try table format
    print("Bold markdown format parsing failed, trying table format...")
    df = parse_table_fallback(response_text)
    if not df.empty:
        print(f"Table format parsing succeeded with {len(df)} rows")
        return df

    # If table format failed, try bullet format
    print("Table format parsing failed, trying bullet format...")
    df = parse_bullet_format(response_text)
    if not df.empty:
        print(f"Bullet format parsing succeeded with {len(df)} rows")
        return df

    # All methods failed
    print("All parsing methods failed - no valid data to create DataFrame")
    print("\nSample of model response (first 10 lines):")
    for i, line in enumerate(response_text.split("\n")[:10], 1):
        print(f"{i:02d}: {line}")
    return pd.DataFrame()

# ---------------------------
# Orchestration
# ---------------------------

def compile_all_responses() -> pd.DataFrame:
    """Compile multiple API responses into a single DataFrame."""
    all_responses = []
    conversation_history = None

    # Get initial response
    initial_response, conversation_history = get_securities_laws()
    if initial_response:
        print("\nInitial response:")
        print("="*50)
        print(initial_response)
        print("="*50)
        print(f"Response length: {len(initial_response)} characters")
        all_responses.append(initial_response)
    else:
        print("ERROR: No initial response received from API!")
        return pd.DataFrame()

    # Follow-up prompts for additional laws (unchanged)
    follow_up_prompts = [
        """Starting with number {last_num}, list 20 more federal securities laws using this exact format for each:
Date: YYYY-MM-DD
Title: [title]
Authority: [body]
Description: [brief]
Impact: [impact]
Litigation Risk: Yes/No
Corporate Governance: Yes/No
Proprietary Costs: Yes/No
Information Asymmetry: Yes/No
Unsophisticated Investors: Yes/No
Equity Issuance: Yes/No
Reputation Risk: Yes/No
References: [link]""",

        "Continue from number {last_num}. Provide 20 more laws using the exact same format.",
        
        "List 20 more laws starting at number {last_num}. Use the same format.",
        
        "Add 20 more laws beginning with number {last_num}. Keep the same format.",
        
        "Provide 20 more laws from number {last_num}. Same format.",
        
        "Recall that you have to identify at least 100 federal securities laws. Recall securities regulation is the field of U.S. law that covers transactions and other dealings with securities. Securities laws aim at ensuring that investors receive accurate and necessary information regarding the type and value of the interest under consideration for purchase."
    ]
    last_num = len(parse_response_to_dataframe(initial_response)) + 1

    for i, prompt_template in enumerate(follow_up_prompts, 1):
        prompt = prompt_template.format(last_num=last_num)
        conversation_history = add_follow_up_prompt(conversation_history, prompt)
        response, conversation_history = get_securities_laws(conversation_history)

        if response:
            print(f"\nFollow-up response {i}:")
            print(response)
            all_responses.append(response)
            df = parse_response_to_dataframe(response)
            last_num += len(df)

    # Parse all responses into DataFrames and concatenate
    dfs = []
    for response in all_responses:
        df = parse_response_to_dataframe(response)
        if not df.empty:
            dfs.append(df)

    if not dfs:
        print("No valid data frames were created!")
        return pd.DataFrame()

    # Concatenate all DataFrames
    final_df = pd.concat(dfs, ignore_index=True)

    # Remove duplicates using multiple fields to better identify unique laws
    final_df['Title_clean'] = final_df['Regulation Title'].fillna('').str.lower().str.strip()
    final_df['Description_clean'] = final_df['Description'].fillna('').str.lower().str.strip()

    # Create composite key for deduplication
    final_df['dedup_key'] = final_df.apply(
        lambda row: f"{row['Date']}_{row['Title_clean']}_{row['Description_clean'][:50]}",
        axis=1
    )

    final_df = final_df.drop_duplicates(subset=['dedup_key'], keep='first')
    final_df = final_df.drop(['Title_clean', 'Description_clean', 'dedup_key'], axis=1)

    # Sort by date
    try:
        final_df['DateSort'] = pd.to_datetime(final_df['Date'], errors='coerce')
        final_df = final_df.dropna(subset=['DateSort'])
        final_df = final_df.sort_values('DateSort', ascending=False)
        final_df = final_df.drop('DateSort', axis=1)
    except Exception as e:
        print(f"Warning: Could not sort by date due to: {e}")
        print("Problematic dates:")
        print(final_df['Date'].value_counts())

    # Return the final DataFrame
    return final_df


if __name__ == "__main__":
    # Compile all responses into a DataFrame
    df = compile_all_responses()

    if df.empty:
        print("\nError: No data was collected!")
    else:
        # Display basic statistics
        print(f"\nTotal number of unique laws: {len(df)}")
        print("\nMost recent laws:")
        print(df.head().to_string())

        # Save to CSV (path can be changed as needed)
        output_path = 'enter file path here'
        df.to_csv(output_path, index=False)
        print(f"\nDatabase saved to: {output_path}")
        
# 2. Add column for Year 

import pandas as pd

df=pd.read_csv('enter file path here')

# Clean parentheses and dashes from text columns
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].str.replace('(', '').str.replace(')', '').str.replace('-', '')
    
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df

#Excluding years prior to 2002 2020 and 2021 since we don't have forecast data. We also exclude years 2018, 2019 from law file because we need 2 years after and we have data
#up to 2019
filtered_df = df[~df['Year'].isin([1986, 1987, 1988, 1989,1990, 1991, 1992, 1993, 1994, 1995, 1996,
                                   1997, 1998, 1999, 2000, 2001,2018, 2019, 2020, 2021, 2022, 2023, 2024])]

filtered_df_with_titles = filtered_df.dropna(subset=["Regulatory Body"])

filtered_df_with_titles.to_csv('enter file path here')